<center><a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a></center>





# 4b. 部署您的模型

現在我們已經有了一個訓練良好的模型，是時候使用它了。在這個練習中，我們將向模型展示新的圖像，並檢測手語字母表中的正確字母。讓我們開始吧！


## 4b.1 目標(Objectives)

-   從磁碟載入已訓練好的模型
-   這模型是用不同格式的圖像資料訓練的，因此我們需重新調整我們的圖像格式
-   使用模型從未見過的新圖像進行推論(inference)並評估其性能

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
import torchvision.io as tv_io
import torchvision.transforms.v2 as transforms
import torchvision.transforms.functional as F
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()


## 4b.2 載入模型

現在我們進入了一個新的 notebook，讓我們載入我們先前訓練好並儲存起來模型。我們在上一個練習中建立了一個名為「asl_model」的資料夾。我們可以通過選擇相同的資料夾來載入模型。


由於我們的模型使用了[自定義模組](https://pytorch.org/tutorials/beginner/examples_nn/two_layer_net_module.html)，我們需要載入該類別(Class)的程式碼。我們在 [utils.py](./uitls.py) 中保存了該程式碼的副本。

In [ ]:
from utils import MyConvBlock


現在我們已經有了 `MyConvBlock` 的定義，我們可以使用 [torch.load](https://pytorch.org/docs/stable/generated/torch.load.html) 從路徑載入模型。我們可以使用 `map_location` 來指定裝置。當我們印出(print)模型時，它看起來與上一個 notebook 中的相同嗎？

In [ ]:
model = torch.load('model.pth', map_location=device)
model


我們也可以驗證模型是否在我們的 GPU 上。

In [ ]:
next(model.parameters()).device


## 4b.3 為模型準備圖像


現在是時候使用模型對它從未見過的新圖像進行預測了。這也被稱為推論(inference)。我們在 `data/asl_images` 資料夾中有一組圖像。嘗試使用左側navigation欄打開它並探索這些圖像。

您會注意到，我們擁有的圖像比我們資料集中的圖像解析度高得多。它們也是彩色的。請記住，我們資料集中的圖像是 28x28 像素的灰階圖像。重要的是要記住，每當我們使用模型進行預測時，輸入必須與模型訓練時的資料形狀相匹配。對於這個模型，訓練資料集的形狀是：(27455, 28, 28, 1)。這對應於 27455 張圖像，每張 28x28 像素，每個有一個顏色通道(channel)（灰階）。


### 4b.3.1 顯示圖像


當我們使用模型對新圖像進行預測時，顯示圖像會很有用。我們可以使用 matplotlib 函式庫來做到這一點。

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def show_image(image_path):
    image = mpimg.imread(image_path)
    plt.imshow(image, cmap='gray')

In [ ]:
show_image('data/asl_images/b.png')


### 4b.3.2 縮放圖像


我們資料集中的圖像是 28x28 像素的灰階圖像。我們需要確保為預測方法傳遞相同大小和灰階的圖像。有幾種方法可以使用 Python 編輯圖像，但 TorchVision 也有 [read_image](https://pytorch.org/vision/stable/generated/torchvision.io.read_image.html) 函式。我們可以通過 [ImageReadMode](https://pytorch.org/vision/stable/generated/torchvision.io.ImageReadMode.html#torchvision.io.ImageReadMode) 讓它知道要讀取什麼類型的圖像。

In [ ]:
image = tv_io.read_image('data/asl_images/b.png', tv_io.ImageReadMode.GRAY)
image


讓我們看看圖像的形狀。

In [ ]:
image.shape


這個圖像比我們訓練的要大得多。我們可以再次使用 [TorchVision](https://pytorch.org/vision/stable/index.html) 的 [Transforms](https://pytorch.org/vision/0.9/transforms.html) 來獲得符合我們模型期望的格式的資料。

我們將：

-   使用 [ToDtype](https://pytorch.org/vision/stable/generated/torchvision.transforms.v2.ToDtype.html) 將圖像轉換為浮點數(float)
    -   我們將設置 `scale` 為 `True` 以將[0, 255] 轉換為[0, 1]

-   使用 [Resize](https://pytorch.org/vision/stable/generated/torchvision.transforms.v2.Resize.html#torchvision.transforms.v2.Resize) 將圖像調整為 28 x 28 像素

-   使用 [Grayscale](https://pytorch.org/vision/stable/generated/torchvision.transforms.v2.Grayscale.html#torchvision.transforms.v2.Grayscale) 將圖像轉換為灰階
    -   這一步實際上沒有做任何事情，因為我們的模型已經是灰階的，但我們在這裡添加它是為了展示獲取灰階圖像的另一種方法。

In [ ]:
IMG_WIDTH = 28
IMG_HEIGHT = 28

preprocess_trans = transforms.Compose([
    transforms.ToDtype(torch.float32, scale=True), # Converts [0, 255] to [0, 1]
    transforms.Resize((IMG_WIDTH, IMG_HEIGHT)),
    transforms.Grayscale()  # From Color to Gray
])


讓我們在一張圖像上測試 `preprocess_trans` 以確保它正常工作：

In [ ]:
processed_image = preprocess_trans(image)
processed_image


數字看起來正確，但形狀如何？

In [ ]:
processed_image.shape

接下來，讓我們繪製圖像，看看它是否看起來像我們訓練的圖像。

In [ ]:
plot_image = F.to_pil_image(processed_image)
plt.imshow(plot_image, cmap='gray')

看起來不錯！讓我們將它傳遞給我們的模型。

### 4b.4 進行預測


好的，現在我們準備進行預測了！我們的模型仍然期望一批圖像。如果 [squeeze](https://pytorch.org/docs/stable/generated/torch.squeeze.htmlhttps://pytorch.org/docs/stable/generated/torch.squeeze.html) 移除維度為 1 的維度，[unsqueeze](https://pytorch.org/docs/stable/generated/torch.unsqueeze.htmlhttps://pytorch.org/docs/stable/generated/torch.unsqueeze.html) 在我們指定的索引處添加一個維度為 1 的維度。第一個維度通常是批次維度(batch dimension)，所以我們可以說 `.unsqueeze(0)`。

In [ ]:
batched_image = processed_image.unsqueeze(0)
batched_image.shape


接下來，我們應該確保輸入張量(tensor)與模型(model)在相同的 `device` 上。

In [ ]:
batched_image_gpu = batched_image.to(device)
batched_image_gpu.device


現在我們準備將它傳遞給模型！

In [ ]:
output = model(batched_image_gpu)
output

### 4b.4.1 理解預測

預測的格式是一個長度為 24 的陣列。值越大，輸入圖像屬於相應類別的可能性就越大。讓我們使它更易讀一些。我們可以從找出陣列中代表最高概率的元素開始。這可以使用 numpy 函式庫和 [argmax](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) 函式輕鬆完成。

In [ ]:
prediction = output.argmax(dim=1).item()
prediction


預測陣列的每個元素代表手語字母表中的一個可能字母。請記住，j 和 z 不是選項，因為它們涉及移動手部，而我們只處理靜態照片。讓我們創建一個預測陣列索引與相應字母之間的映射。

In [ ]:
# Alphabet does not contain j or z because they require movement
alphabet = "abcdefghiklmnopqrstuvwxy"


現在我們可以傳入我們的預測索引來找到相應的字母。

In [ ]:
alphabet[prediction]

#### 練習：將所有內容整合在一起


讓我們將所有內容放在一個函式中，這樣我們就可以僅從圖像文件進行預測。使用上面的函式和步驟在下面的函式中實現它。如果您需要幫助，可以點擊下面的三個點來查看解決方案。

In [ ]:
def predict_letter(file_path):
    # Show image
    FIXME
    # Load and grayscale image
    image = FIXME
    # Transform image
    image = FIXME
    # Batch image
    image = FIXME
    # Send image to correct device
    image = FIXME
    # Make prediction
    output = FIXME
    # Find max index
    prediction = FIXME
    # Convert prediction to letter
    predicted_letter = FIXME
    # Return prediction
    return predicted_letter

#### 解決方案

點擊下面的 '...' 查看解決方案。

In [ ]:
# SOLUTION
def predict_letter(file_path):
    show_image(file_path)
    image = tv_io.read_image(file_path, tv_io.ImageReadMode.GRAY)
    image = preprocess_trans(image)
    image = image.unsqueeze(0)
    image = image.to(device)
    output = model(image)
    prediction = output.argmax(dim=1).item()
    # convert prediction to letter
    predicted_letter = alphabet[prediction]
    return predicted_letter

In [ ]:
predict_letter("data/asl_images/b.png")


讓我們也使用該函式處理 asl\_images 資料集中的 'a' 字母：

In [ ]:
predict_letter("data/asl_images/a.png")

## 4b.5 總結

這些練習做得很好！您已經完成了從頭開始訓練高精度模型的完整過程，然後使用該模型進行新的及有功用的預測。如果您有一些時間，我們鼓勵您使用網絡攝像頭拍照，通過將它們拖放到 data/asl_images 資料夾中上傳，並在它們上測試模型。對於 Mac，您可以使用 Photo Booth軟體。對於 Windows，您可以從開始選單去選擇相機應用程式。我們希望您嘗試一下。這是學習一些手語的好機會！例如，嘗試您名字中的字母。

我們可以想像這個模型如何用於教某人手語的應用程式，甚至幫助不能說話的人與電腦互動。

### 4b.5.1 清空記憶體


在繼續之前，請執行以下程式碼區塊(cell)以清空 GPU 記憶體。

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### 4b.5.2 下一步

我們希望您喜歡這些練習。在接下來的部分中，我們將學習如何在沒有強大資料集的情況下利用深度學習。我們下次見！
要了解更多關於在邊緣裝置(edge)上推論(inference)的資訊，請查看[這篇論文](http://web.eecs.umich.edu/~mosharaf/Readings/FB-ML-Edge.pdf)。



現在我們已經熟悉了構建自己的模型並對它們的工作原理有了一些了解，我們將把注意力轉向使用預訓練模型(pre-trained models)這一個非常強大的技術。

<center><a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a></center>


